In [82]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
import time
import random
import json

We get the id of the gene list, in order to request the expansion to gene@home

In [75]:
gene_list = []
final = {}

with open("final_list_genes_merged.txt","r") as f:
    gene_list = f.read().splitlines()
    

fantom_db = pd.read_csv("anno-fantom.csv")
snoDB = pd.read_csv("snoDB_All_V2.0.csv",keep_default_na=False)
    

In [76]:
def get_unexpanded_genes(gene_list):
    url = 'https://gene.disi.unitn.it/test/gene_history-z.php'
    data = {
        'org': 'Hs+(Homo+sapiens)',
        'tsize': '',
        'iter': '',
        'alpha': '0.05',
        'lgn': '',
        'submit': 'SEARCH'}
    final_list = []
    for gene in gene_list:
        data['lgn'] = gene
        soup = BeautifulSoup(requests.post(url, data = data).content,"html.parser")

        if soup.find("div",{"id":"dialog-end"}) != None:
            final_list.append(gene)
        
        time.sleep(random.uniform(1.5,2.0))

    return final_list

gene_synonyms = {}
for gene in gene_list:
    
    ids = fantom_db.loc[fantom_db['n4'] == gene.upper()]['id'].tolist()
    if(ids != []):
        final[gene.upper()] = ids
    else:
        if(gene.upper() not in gene_synonyms):
            #not found, we look for synonyms
            #controlla host_name 
            data = snoDB.loc[snoDB['host_gene_name'] == gene.upper()]
            if(not(data.empty)):
                #if we find host gene name , find for synonyms
                host_synonyms = data.loc[data['host_synonyms'] != ""]['host_synonyms']
                if(not(host_synonyms.empty)):
                    synonyms = ';'.join(host_synonyms.tolist())
                    synonyms = synonyms.split(";")


                    to_append = [False,synonyms]
                    
                    synonyms = get_unexpanded_genes(synonyms)

                    for synonym in synonyms:
                        ids = fantom_db.loc[fantom_db['n4'] == synonym.upper()]['id'].tolist()
                        if(ids != []):
                            final[synonym.upper()] = ids
            
            data = snoDB.loc[snoDB['gene_name'] == gene.upper()]
            if(not(data.empty)):
                sno_synonyms = data.loc[data['synonyms'] != ""]['synonyms']

                if(not(sno_synonyms.empty)):

                    synonyms = ';'.join(sno_synonyms.tolist())
                    synonyms = synonyms.split(";")
                    
                    synonyms = get_unexpanded_genes(synonyms)

                    to_append = [False,synonyms]

                    for synonym in synonyms:
                        ids = fantom_db.loc[fantom_db['n4'] == synonym.upper()]['id'].tolist()
                        if(ids != []):
                            final[synonym.upper()] = ids
            gene_synonyms[gene.upper()] = to_append


        #controlla snorna name

In [83]:
gene_synonyms
with open('synonyms.json', 'w') as f:
    json.dump(gene_synonyms, f)

In [78]:

import pprint
pprint.pprint(final)

{'CDKN2B-AS1': ['T192270',
                'T192274',
                'T192276',
                'T192277',
                'T192290',
                'T192291'],
 'CWF19L1': ['T000106', 'T000107'],
 'EIF4A1': ['T063635'],
 'EP400': ['T023102', 'T023103'],
 'HIF1A-AS2': ['T039389', 'T039390'],
 'MEG8': ['T035291'],
 'NOP56': ['T099399', 'T099400'],
 'RACK1': ['T153228',
           'T153229',
           'T153230',
           'T153232',
           'T153234',
           'T153235',
           'T153237',
           'T153238',
           'T153239',
           'T153240',
           'T153241',
           'T153242',
           'T153243',
           'T153244',
           'T153245',
           'T153246'],
 'RPL13A': ['T076769', 'T076772'],
 'RPS13': ['T012233', 'T012234'],
 'SCARNA12': ['T028812', 'T028813'],
 'SNHG12': ['T093143'],
 'SNHG20': ['T063682'],
 'SNHG5': ['T168212'],
 'SNHG6': ['T186052'],
 'SNHG7': ['T191363'],
 'SNHG8': ['T139228', 'T139229'],
 'SNORA1': ['T019615'],
 'SNORA12': ['T

In [79]:
missing_genes = []

for key in final:
    if(final[key] == []):
        missing_genes.append(key)

print(len(missing_genes))
print(missing_genes)

0
[]


In [94]:
list_to_expand = []
for key in final:
    list_to_expand.append([key,final[key]])
list_to_expand
with open("genes_to_expand_final.txt","w") as f:
    for id in list_to_expand:
        _lista = "\n".join(id[1])
        f.write(f"{_lista}\n")

with open("genes_to_expand_final-labeled.txt","w") as f:
    for id in list_to_expand:
        f.write(f"{id[0]} - {'; '.join(id[1])}\n")

In [86]:
list_to_expand

['CDKN2B-AS1',
 ['T192270', 'T192274', 'T192276', 'T192277', 'T192290', 'T192291'],
 'CWF19L1',
 ['T000106', 'T000107'],
 'EIF4A1',
 ['T063635'],
 'EP400',
 ['T023102', 'T023103'],
 'HIF1A-AS2',
 ['T039389', 'T039390'],
 'MEG8',
 ['T035291'],
 'NOP56',
 ['T099399', 'T099400'],
 'RACK1',
 ['T153228',
  'T153229',
  'T153230',
  'T153232',
  'T153234',
  'T153235',
  'T153237',
  'T153238',
  'T153239',
  'T153240',
  'T153241',
  'T153242',
  'T153243',
  'T153244',
  'T153245',
  'T153246'],
 'RPL13A',
 ['T076769', 'T076772'],
 'RPS13',
 ['T012233', 'T012234'],
 'SCARNA12',
 ['T028812', 'T028813'],
 'SNHG12',
 ['T093143'],
 'SNHG20',
 ['T063682'],
 'SNHG5',
 ['T168212'],
 'SNHG6',
 ['T186052'],
 'SNHG7',
 ['T191363'],
 'SNHG8',
 ['T139228', 'T139229'],
 'SNORA1',
 ['T019615'],
 'SNORA12',
 ['T000102'],
 'SNORA14B',
 ['T091265', 'T091266'],
 'SNORA16A',
 ['T093142'],
 'SNORA21',
 ['T057724'],
 'SNORA23',
 ['T019697'],
 'SNORA24',
 ['T139230'],
 'SNORA32',
 ['T019614'],
 'SNORA38',
 ['T1

In [81]:
len(list_to_expand)

176